**Optimize geometry of LiH**

Follows Pennylane adaptive circuit tutorial to obtain circuit, and molecular geometry tutorial to optimize geometry

In [1]:
!pip install pennylane &> /dev/null

In [2]:
!pip install pennylane-qchem &> /dev/null

In [3]:
from pennylane import numpy as np
import pennylane as qml

In [6]:
from pennylane import qchem

In [4]:
# Set up the molecule. Geometry coordinates from CCCBDB

symbols = ["Li", "H"]
geometry = np.array([0.0,0.0,0.0,0.0,0.0,3.064], requires_grad=True)

Now create the circuit using https://pennylane.ai/qml/demos/tutorial_adaptive_circuits.html

First - compute the Hamiltonian and calculate the single and double electron excitations (maybe also try looking at 3 & 4 excitations to see what effect it has on accuracy?).

In [7]:
active_electrons = 2
active_orbitals = 5

H, qubits = qchem.molecular_hamiltonian(
    symbols,
    geometry,
    active_electrons=active_electrons, # figure out how to get this
    active_orbitals=active_orbitals # figure out how to get this
)

singles, doubles = qchem.excitations(active_electrons, qubits)

In [8]:
print(qubits)

10


In [9]:
print(singles)

[[0, 2], [0, 4], [0, 6], [0, 8], [1, 3], [1, 5], [1, 7], [1, 9]]


In [11]:
print(doubles)

[[0, 1, 2, 3], [0, 1, 2, 5], [0, 1, 2, 7], [0, 1, 2, 9], [0, 1, 3, 4], [0, 1, 3, 6], [0, 1, 3, 8], [0, 1, 4, 5], [0, 1, 4, 7], [0, 1, 4, 9], [0, 1, 5, 6], [0, 1, 5, 8], [0, 1, 6, 7], [0, 1, 6, 9], [0, 1, 7, 8], [0, 1, 8, 9]]


Get the Hartree-Fock state

In [12]:
hf_state = qchem.hf_state(active_electrons, qubits)
print(hf_state)

[1 1 0 0 0 0 0 0 0 0]


Build the circuit:



1.   Compute gradients for all double excitations
2.   Select the double excitations with 
gradients larger than a pre-defined threshold

1.   Select the double excitations with 
gradients larger than a pre-defined threshold
2.   Perform VQE to obtain the optimized parameters for the selected double excitations

1.   Repeat steps 1 and 2 for the single excitations

In [13]:
# This function builds the circuit based on the set of parameters passed below
# Wires - i.e. qubits in the circuit, and specific qubit excitations

def circuit_1(params, wires, excitations):

    qml.BasisState(hf_state, wires=wires)

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4: #double excitations have four entries
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation) #two entries each

In [14]:
# specify the device
dev = qml.device("default.qubit", wires=qubits)

# cost function calculate the expectation value of the Hamiltonian based on the
# ansatz which is the circuit
cost_fn = qml.ExpvalCost(circuit_1, H, dev, optimize=True)

# # get the gradient of the cost function
circuit_gradient = qml.grad(cost_fn, argnum=0)

# # initialize parameters to zero so gradients computed from HF state
params = [0.0] * len(doubles)

# #calculate the gradient for the double excitations
# this one actually calls the circuit_1 function
grads = circuit_gradient(params, excitations=doubles)

for i in range(len(doubles)):
    print(f"Excitation : {doubles[i]}, Gradient: {grads[i]}")

Excitation : [0, 1, 2, 3], Gradient: -0.013282692971956422
Excitation : [0, 1, 2, 5], Gradient: -2.1684043449710084e-19
Excitation : [0, 1, 2, 7], Gradient: 3.2526065174565114e-19
Excitation : [0, 1, 2, 9], Gradient: 0.03486133571916371
Excitation : [0, 1, 3, 4], Gradient: 4.065758146820643e-20
Excitation : [0, 1, 3, 6], Gradient: -1.3552527156068803e-19
Excitation : [0, 1, 3, 8], Gradient: -0.034861335719163965
Excitation : [0, 1, 4, 5], Gradient: -0.02330625477597942
Excitation : [0, 1, 4, 7], Gradient: 0.0
Excitation : [0, 1, 4, 9], Gradient: 0.0
Excitation : [0, 1, 5, 6], Gradient: 0.0
Excitation : [0, 1, 5, 8], Gradient: 2.1684043449710118e-19
Excitation : [0, 1, 6, 7], Gradient: -0.02330625477597944
Excitation : [0, 1, 6, 9], Gradient: 0.0
Excitation : [0, 1, 7, 8], Gradient: -1.8973538018496287e-19
Excitation : [0, 1, 8, 9], Gradient: -0.12416373075295714


In [15]:
# Just keep the double excitations that have gradients > 10^-5

doubles_select = [doubles[i] for i in range(len(doubles)) if abs(grads[i]) > 1.0e-5]
doubles_select

[[0, 1, 2, 3],
 [0, 1, 2, 9],
 [0, 1, 3, 8],
 [0, 1, 4, 5],
 [0, 1, 6, 7],
 [0, 1, 8, 9]]

In [16]:
# Calculate the parameters (params_doubles) for each of the double excitation gates
# using gradient descent to find the lowest expectation value of H on circuit_1
# having applied the selected double excitations

opt = qml.GradientDescentOptimizer(stepsize=0.5)

params_doubles = np.zeros(len(doubles_select), requires_grad=True)

for n in range(20):
    params_doubles = opt.step(cost_fn, params_doubles, excitations=doubles_select)

In [17]:
# Create new circuit that applies the selected gates based on the gradients
# determined above gradient threshold and pass those gates the parameters
# calculated in the GDO process

def circuit_2(params, wires, excitations, gates_select, params_select):
    qml.BasisState(hf_state, wires=wires)

    # apply the selected gates and apply parameters
    
    for i, gate in enumerate(gates_select):
        if len(gate) == 4:
            qml.DoubleExcitation(params_select[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params_select[i], wires=gate)

    for i, gate in enumerate(excitations):
        if len(gate) == 4:
            qml.DoubleExcitation(params[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params[i], wires=gate)

In [18]:
# Now compute gradients for single excitations, in same way we did for the
# double excitations

# cost function calculate the expectation value of the Hamiltonian based on the
# ansatz which is the circuit
cost_fn = qml.ExpvalCost(circuit_2, H, dev, optimize=True)

# get the gradient of the cost function
circuit_gradient = qml.grad(cost_fn, argnum=0)

# initialize parameters to zero so gradients computed from HF state
# note that we're only doing this for the single excitations as the 
# doubles are already fixed
params = [0.0] * len(singles)

# calculate the gradient for the single excitations, but with the previously
# computed double excitations held constant. Note that circuit_2 takes the
# following parameters: params, wires, excitations, gates_select, params_select

grads = circuit_gradient(
    params,
    excitations=singles,
    gates_select = doubles_select,
    params_select = params_doubles)

for i in range(len(singles)):
    print(f"Excitation : {singles[i]}, Gradient: {grads[i]}")

Excitation : [0, 2], Gradient: -0.005146190384624545
Excitation : [0, 4], Gradient: -1.4675863503195058e-17
Excitation : [0, 6], Gradient: 7.690800194090734e-19
Excitation : [0, 8], Gradient: -0.000980381946160425
Excitation : [1, 3], Gradient: 0.004997627978515387
Excitation : [1, 5], Gradient: -5.784431962375564e-19
Excitation : [1, 7], Gradient: -3.0538143873782976e-18
Excitation : [1, 9], Gradient: 0.0015198027890503626


In [19]:
singles_select = [singles[i] for i in range(len(singles)) if abs(grads[i]) > 1.0e-5]
singles_select

[[0, 2], [0, 8], [1, 3], [1, 9]]

In [20]:
excitations = doubles_select + singles_select
excitations

[[0, 1, 2, 3],
 [0, 1, 2, 9],
 [0, 1, 3, 8],
 [0, 1, 4, 5],
 [0, 1, 6, 7],
 [0, 1, 8, 9],
 [0, 2],
 [0, 8],
 [1, 3],
 [1, 9]]

In [21]:
cost_fn = qml.ExpvalCost(circuit_1, H, dev, optimize=True)

params = np.zeros(len(doubles_select + singles_select), requires_grad=True)

gates_select = doubles_select + singles_select

for n in range(20):
    params, energy = opt.step_and_cost(cost_fn, params, excitations=gates_select)
    print("n = {:},  E = {:.8f} H".format(n, energy))

n = 0,  E = -7.86109478 H
n = 1,  E = -7.86949977 H
n = 2,  E = -7.87434711 H
n = 3,  E = -7.87715002 H
n = 4,  E = -7.87878428 H
n = 5,  E = -7.87974964 H
n = 6,  E = -7.88032993 H
n = 7,  E = -7.88068655 H
n = 8,  E = -7.88091179 H
n = 9,  E = -7.88105877 H
n = 10,  E = -7.88115837 H
n = 11,  E = -7.88122874 H
n = 12,  E = -7.88128070 H
n = 13,  E = -7.88132074 H
n = 14,  E = -7.88135286 H
n = 15,  E = -7.88137955 H
n = 16,  E = -7.88140237 H
n = 17,  E = -7.88142234 H
n = 18,  E = -7.88144014 H
n = 19,  E = -7.88145621 H


In [22]:
params

tensor([ 0.0530969 , -0.1060742 ,  0.10604959,  0.05275959,  0.05277617,
         0.2361398 ,  0.02649252,  0.00483553, -0.02614315, -0.00459074], requires_grad=True)

**Now we do the geometry and energy optimization**


In [23]:
x = np.array([0.0,0.0,0.0,0.0,0.0,3.064], requires_grad=True)

In [24]:
# Get the Hamiltonian for the molecule, based on x (position of nuclei)
# This is different from the earlier code where we weren't changing positions

def H(x):
    return qml.qchem.molecular_hamiltonian(symbols, x, charge=0)[0]

In [25]:
# we made the HF state earlier, doing it here again

hf = qchem.hf_state(active_electrons, qubits)
print(hf)

[1 1 0 0 0 0 0 0 0 0]


In [26]:
print(qubits)

10


In [27]:
num_wires = qubits
dev = qml.device("default.qubit", wires=num_wires)
print(num_wires)

10


In [28]:
@qml.qnode(dev)
def circuit(params, obs, wires):
    qml.BasisState(hf, wires=wires)

    #added this code, replacing hard-coded excitations in tutorial

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation) 
    
    return qml.expval(obs)

In [29]:
# Get the expectation value of the Hamiltonian, based on the excitation
# parameters and the x (position) value. A call to 'circuit' returns
# the expectation value of measuring the Hamiltonian

# NOTE: find a way to not have to pass the excitations each time?

def cost(params, x):
    hamiltonian = H(x)
    return circuit(params, obs=hamiltonian, wires=range(num_wires))

Explanation of [qml.finite_diff](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.finite_diff.html#pennylane.finite_diff)

In [30]:
# Find the partial differential of H w.r.t. x

def grad_x(params, x):
    grad_h = qml.finite_diff(H)(x)
    grad = [circuit(params, obs=obs, wires=range(num_wires)) for obs in grad_h]
    return np.array(grad)

In [31]:
# Experiment with different step sizes?

opt_theta = qml.GradientDescentOptimizer(stepsize=0.4)
opt_x = qml.GradientDescentOptimizer(stepsize=0.8)

In [32]:
theta = np.zeros(len(excitations), requires_grad=True)
len(theta)

10

In [33]:
from functools import partial

# store the values of the cost function
energy = []

# store the values of the bond length
bond_length = []

# Factor to convert from Bohrs to Angstroms
bohr_angs = 0.529177210903

In [34]:
for n in range(100):

    # Optimize the circuit parameters
    theta.requires_grad = True
    x.requires_grad = False
    theta, _ = opt_theta.step(cost, theta, x)

    # Optimize the nuclear coordinates
    x.requires_grad = True
    theta.requires_grad = False
    _, x = opt_x.step(cost, theta, x, grad_fn=grad_x)

    energy.append(cost(theta, x))
    bond_length.append(np.linalg.norm(x[0:3] - x[3:6]) * bohr_angs)

    if n % 4 == 0:
        print(f"Step = {n},  E = {energy[-1]:.8f} Ha,  bond length = {bond_length[-1]:.5f} A")

    # Check maximum component of the nuclear gradient
    if np.max(grad_x(theta, x)) <= 1e-05:
        break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" "Ground-state equilibrium geometry")
print("%s %4s %8s %8s" % ("symbol", "x", "y", "z"))
for i, atom in enumerate(symbols):
    print(f"  {atom}    {x[3 * i]:.4f}   {x[3 * i + 1]:.4f}   {x[3 * i + 2]:.4f}")

WireError: ignored

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
fig.set_figheight(5)
fig.set_figwidth(12)

# Add energy plot on column 1
E_fci = -1.27443765658
E_vqe = np.array(energy)
ax1 = fig.add_subplot(121)
ax1.plot(range(n+1), E_vqe-E_fci, 'go-', ls='dashed')
ax1.plot(range(n+1), np.full(n+1, 0.001), color='red')
ax1.set_xlabel("Optimization step", fontsize=13)
ax1.set_ylabel("$E_{VQE} - E_{FCI}$ (Hartree)", fontsize=13)
ax1.text(5, 0.0013, r'Chemical accuracy', fontsize=13)
plt.yscale("log")
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add bond length plot on column 2
d_fci = 0.986
ax2 = fig.add_subplot(122)
ax2.plot(range(n+1), bond_length, 'go-', ls='dashed')
ax2.plot(range(n+1), np.full(n+1, d_fci), color='red')
ax2.set_ylim([0.965,0.99])
ax2.set_xlabel("Optimization step", fontsize=13)
ax2.set_ylabel("bond length ($\AA$)", fontsize=13)
ax2.text(5, 0.9865, r'Equilibrium bond length', fontsize=13)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.subplots_adjust(wspace=0.3)
plt.show()